# 네이버 영화 리뷰 감성 분류
- LSTM 활용
- 출처: WikiDocs
- 데이터 다운로드 링크: https://github.com/e9t/nsmc/

In [1]:
# Konlpy 설치
!pip install Konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 50.0MB/s 
     |████████████████████████████████| 92kB 13.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
import konlpy
konlpy.__version__

'0.5.2'

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [5]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ratings_train.txt to ratings_train.txt


In [6]:
uploaded = files.upload()
testfile = list(uploaded.keys())[0]

Saving ratings_test.txt to ratings_test.txt


### 데이터 전처리

In [7]:
import pandas as pd

train_df= pd.read_table(filename)
test_df= pd.read_table(testfile)

In [8]:
train_df.shape

(150000, 3)

In [9]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [10]:
# 중복 여부 확인
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [11]:
# Null 값 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [12]:
# 긍정, 부정 레이블 값의 갯수
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [13]:
# 중복제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [14]:
# Null 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

### 한글 텍스트 전처리

In [15]:
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
# 공백을 null으로 만듬
train_df['document'].replace('', np.nan, inplace=True)
train_df.isnull().sum()

id            0
document    391
label         0
dtype: int64

In [17]:
train_df = train_df.dropna(how = 'any')
train_df.shape

(145791, 3)

- 텍스트 데이터 셋

In [18]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'].replace('', np.nan, inplace=True)
test_df.isnull().sum()

id            0
document    162
label         0
dtype: int64

In [19]:
test_df = test_df.dropna(how = 'any')
test_df.shape

(48995, 3)

### 한글 형태소 분석

In [20]:
from konlpy.tag import Okt
import tqdm.notebook as tn

In [21]:
# 토큰화와 불용어 제거

stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

X_train=[]
for sentence in tn.tqdm(train_df['document']):
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [22]:
X_test=[]
for sentence in tn.tqdm(test_df['document']):
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [23]:
X_train[:3]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]

### 케라스 인코딩

In [24]:
# 정수 인코딩
max_words = 35000
tokenizer = Tokenizer(num_words=max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [25]:
X_train[:3]

[[50, 454, 16, 260, 659],
 [933, 457, 41, 602, 1, 214, 1449, 24, 961, 675, 19],
 [386, 2444, 25015, 2315, 5671, 2, 222, 9]]

In [26]:
# 전체 데이터의 길이 분포
print('리뷰의 최대 길이 :', max(len(s) for s in X_train))
print('리뷰의 평균 길이 :', sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 69
리뷰의 평균 길이 : 10.911133060339802


In [27]:
#  X_train과 X_test의 모든 샘플의 길이를 동일하게 30으로 셋팅
max_len=30
# 전체 데이터의 길이는 30으로 맞춘다.
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [28]:
y_train = train_df['label'].values
y_test = test_df['label'].values

### LSTM 모델 정의/설정/학습/평가

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

### 1.LSTM

In [103]:
model = Sequential([
    Embedding(max_words, 100),
    LSTM(128),
    Dense(30000, activation='relu'),
    Dense(8000, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, None, 100)         3500000   
_________________________________________________________________
lstm_21 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_61 (Dense)             (None, 30000)             3870000   
_________________________________________________________________
dense_62 (Dense)             (None, 8000)              240008000 
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 8001      
Total params: 247,503,249
Trainable params: 247,503,249
Non-trainable params: 0
_________________________________________________________________


In [104]:
# 모델 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [105]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='naver-lstm-best-model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)

In [106]:
history = model.fit(X_train, y_train, epochs=4, batch_size=60,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/4
1944/1944 [==============================] - 186s 95ms/step - loss: 0.4365 - accuracy: 0.7917 - val_loss: 0.3532 - val_accuracy: 0.8475

Epoch 00001: val_loss improved from inf to 0.35320, saving model to naver-lstm-best-model.h5
Epoch 2/4
1944/1944 [==============================] - 185s 95ms/step - loss: 0.2765 - accuracy: 0.8832 - val_loss: 0.3536 - val_accuracy: 0.8497

Epoch 00002: val_loss did not improve from 0.35320
Epoch 3/4
1944/1944 [==============================] - 185s 95ms/step - loss: 0.2092 - accuracy: 0.9154 - val_loss: 0.3911 - val_accuracy: 0.8468

Epoch 00003: val_loss did not improve from 0.35320
Epoch 4/4
1944/1944 [==============================] - 185s 95ms/step - loss: 0.1600 - accuracy: 0.9364 - val_loss: 0.4388 - val_accuracy: 0.8428

Epoch 00004: val_loss did not improve from 0.35320


In [107]:
# Best Model 선택
from keras.models import load_model
best_model = load_model('naver-lstm-best-model.h5')

In [108]:
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f'Accuracy: {acc:.4f}')

1532/1532 - 10s - loss: 0.3604 - accuracy: 0.8446
Accuracy: 0.8446


### 2. Simple RNN

In [121]:
del model2

In [122]:
from tensorflow.keras.layers import SimpleRNN

model2 = Sequential([
    Embedding(max_words, 100),
    SimpleRNN(128),
    Dense(10000, activation='relu'),
    Dense(300, activation='relu'),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, None, 100)         3500000   
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 128)               29312     
_________________________________________________________________
dense_69 (Dense)             (None, 10000)             1290000   
_________________________________________________________________
dense_70 (Dense)             (None, 300)               3000300   
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 301       
Total params: 7,819,913
Trainable params: 7,819,913
Non-trainable params: 0
_________________________________________________________________


In [123]:
# 모델 설정
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [124]:
checkpointer = ModelCheckpoint(filepath='naver-rnn-best-model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)

In [125]:
history2 = model2.fit(X_train, y_train, epochs=4, batch_size=60,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/4
1944/1944 [==============================] - 100s 51ms/step - loss: 0.4566 - accuracy: 0.7768 - val_loss: 0.3655 - val_accuracy: 0.8418

Epoch 00001: val_loss improved from inf to 0.36551, saving model to naver-rnn-best-model.h5
Epoch 2/4
1944/1944 [==============================] - 99s 51ms/step - loss: 0.3066 - accuracy: 0.8696 - val_loss: 0.3839 - val_accuracy: 0.8396

Epoch 00002: val_loss did not improve from 0.36551
Epoch 3/4
1944/1944 [==============================] - 99s 51ms/step - loss: 0.2255 - accuracy: 0.9084 - val_loss: 0.4162 - val_accuracy: 0.8313

Epoch 00003: val_loss did not improve from 0.36551
Epoch 4/4
1944/1944 [==============================] - 99s 51ms/step - loss: 0.1630 - accuracy: 0.9354 - val_loss: 0.5130 - val_accuracy: 0.8154

Epoch 00004: val_loss did not improve from 0.36551


In [126]:
best_model = load_model('naver-rnn-best-model.h5')
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f'Accuracy: {acc:.4f}')

1532/1532 - 5s - loss: 0.3718 - accuracy: 0.8365
Accuracy: 0.8365


### 3. CNN + LSTM

In [135]:
del model3

In [136]:
from tensorflow.keras.layers import Conv1D,Dropout, MaxPooling1D

model3 = Sequential([
    Embedding(max_words, 100),
    Dropout(0.5),
    Conv1D(64, 5, padding='valid', activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(60),
    Dense(10000, activation='relu'),
    Dense(5000, activation='relu'),
    Dense(200, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, None, 100)         3500000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 60)                30000     
_________________________________________________________________
dense_77 (Dense)             (None, 10000)             610000    
_________________________________________________________________
dense_78 (Dense)             (None, 5000)            

In [137]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='naver-cnn-lstm-best-model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)
history3 = model3.fit(X_train, y_train, epochs=4, batch_size=60,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/4
1944/1944 [==============================] - 90s 45ms/step - loss: 0.5585 - accuracy: 0.6751 - val_loss: 0.4470 - val_accuracy: 0.7768

Epoch 00001: val_loss improved from inf to 0.44700, saving model to naver-cnn-lstm-best-model.h5
Epoch 2/4
1944/1944 [==============================] - 88s 45ms/step - loss: 0.4053 - accuracy: 0.8055 - val_loss: 0.4531 - val_accuracy: 0.7804

Epoch 00002: val_loss did not improve from 0.44700
Epoch 3/4
1944/1944 [==============================] - 87s 45ms/step - loss: 0.3527 - accuracy: 0.8304 - val_loss: 0.4692 - val_accuracy: 0.7788

Epoch 00003: val_loss did not improve from 0.44700
Epoch 4/4
1944/1944 [==============================] - 86s 44ms/step - loss: 0.3120 - accuracy: 0.8491 - val_loss: 0.4889 - val_accuracy: 0.7764

Epoch 00004: val_loss did not improve from 0.44700


In [138]:
best_model = load_model('naver-cnn-lstm-best-model.h5')
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f'Accuracy: {acc:.4f}')

1532/1532 - 5s - loss: 0.4565 - accuracy: 0.7724
Accuracy: 0.7724
